# Data Analysis
Cette étape a pour objectif d’explorer et d’analyser les données contenues dans la base gdelt_benin.db.
Elle consiste à examiner les événements collectés afin d’en extraire des tendances, des patterns ou des anomalies, en s’appuyant sur des techniques d’analyse de données.

Cette phase permet de mieux comprendre le contenu de la base avant d’envisager des analyses plus poussées.

In [ ]:
%pip install --upgrade pandas matplotlib seaborn scikit-learn python-dotenv pmdarima openai ipython-sql
%pip install --upgrade pandas sqlalchemy xgboost plotly dash dash-table

## Identification de la thématique de chaque évènement

Après la création de la base de données gdelt_benin.db, l’un des objectifs est d’identifier la thématique principale de chaque article présent dans la table benin_events.
Le processus suivant, reposant sur l’utilisation d’Azure OpenAI, permet d’analyser le contenu des URL afin d’en extraire le sujet principal abordé dans chaque article.

In [ ]:
import os
import logging
import sqlite3
import pandas as pd
from typing import List, Tuple
from functions.call_openai import call_openai_api
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt, wait_exponential
from concurrent.futures import ThreadPoolExecutor, as_completed

# Chemin vers la base de données
output_dir = "/home/pionner02/Pionner02 UlChris-Project/data"
db_path = os.path.join(output_dir, 'gdelt_benin.db')

# Vérification de l'existence du fichier
if not os.path.exists(db_path):
    print(f"Fichier introuvable : {db_path}")
    exit(1)

# Fonction pour extraire le contenu d'une URL
def extract_url_content(url: str, timeout: int = 5) -> str:
    try:
        response = requests.get(url, timeout=timeout, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            text = ' '.join(p.get_text(strip=True) for p in soup.find_all('p'))[:2000]
            return text if text else "Contenu non pertinent"
        else:
            print(f"URL {url} inaccessible (code {response.status_code})")
            return "Contenu inaccessible"
    except requests.RequestException as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {str(e)}")
        return "Contenu inaccessible"

# Fonction API avec retry
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def call_openai_with_retry(prompt: str, max_tokens: int = 100) -> str:
    return call_openai_api(prompt, max_tokens=max_tokens)

# Fonction pour traiter un événement
def process_event(event: Tuple[int, str, str, float]) -> Tuple[str, int]:
    event_id, source_url, location, tone = event
    print(f"Traitement de l'événement {event_id} - URL : {source_url}")

    content = extract_url_content(source_url)
    prompt = (
        "Tu es un expert en analyse de contenu médiatique spécialisé dans les événements au Bénin. "
        "À partir du contenu extrait de l'URL {source_url}: '{content}', et des informations suivantes : "
        "lieu de l'événement='{location}', ton moyen={tone}, "
        "identifie le thème principal de l'événement décrit. "
        "Le thème doit être une phrase nominale cohérente concise et précise (2 mots au maximum) décrivant précisément le sujet central, "
        "en lien avec le contexte de la publication (ex. 'CONFLIT', 'ECONOMIE', 'SANTE', 'POLITIQUE', 'CRIME', 'CRISE SOCIALE'). "
        "Si le contenu est inaccessible ou insuffisant, utilise le lieu et le ton pour inférer un thème plausible. "
        "Si le contenu est hors sujet (ex. publicité, page d'accueil), retourne 'Contenu non pertinent'. "
        "Aussi recence toujours le thème en majuscule"
        "Les thèmes ne doivent pas avoir un forte connotation péjorative mais informative comme un journaliste "
        "Limite la réponse à 2 mots maximum."
        
    ).format(source_url=source_url, content=content, location=location or "Bénin", tone=tone)

    try:
        theme = call_openai_with_retry(prompt)
        if theme.startswith("Erreur"):
            print(f"Événement {event_id} : {theme}")
            theme = "Thème non identifiable"
    except Exception as e:
        print(f"Erreur API pour l'événement {event_id} : {str(e)}")
        theme = "Thème non identifiable"

    return (theme, event_id)

try:
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("PRAGMA foreign_keys = ON;")

        # Vérifier l'existence de la colonne Themes
        try:
            cursor.execute("ALTER TABLE events ADD COLUMN Themes TEXT;")
            conn.commit()
            print("Colonne Themes ajoutée.")
        except sqlite3.OperationalError:
            print("Colonne Themes existe déjà.")

        # Récupérer les URLs non traitées avec location et tone
        cursor.execute("""
            SELECT GLOBALEVENTID, SOURCEURL, ActionGeo_FullName, AvgTone
            FROM events 
            WHERE SOURCEURL IS NOT NULL AND Themes IS NULL 
            LIMIT 100;  -- Limite pour tester
        """)
        rows: List[Tuple[int, str, str, float]] = cursor.fetchall()
        print(f"{len(rows)} événements à traiter.")

        # Traiter les événements en parallèle
        updates = []
        batch_size = 100
        max_workers = 5  # Ajuster selon les limites de l'API
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_event = {executor.submit(process_event, row): row for row in rows}
            for future in as_completed(future_to_event):
                theme, event_id = future.result()
                updates.append((theme, event_id))
                print(f"Événement {event_id} - Thème : {theme}")

                # Mettre à jour par lots
                if len(updates) >= batch_size:
                    try:
                        cursor.executemany("UPDATE events SET Themes = ? WHERE GLOBALEVENTID = ?", updates)
                        conn.commit()
                        print(f"{len(updates)} événements mis à jour.")
                        updates = []
                    except sqlite3.Error as e:
                        print(f"Erreur lors de la mise à jour : {str(e)}")

        # Mettre à jour les dernières lignes
        if updates:
            try:
                cursor.executemany("UPDATE events SET Themes = ? WHERE GLOBALEVENTID = ?", updates)
                conn.commit()
                print(f"{len(updates)} événements mis à jour.")
            except sqlite3.Error as e:
                print(f"Erreur lors de la mise à jour : {str(e)}")

except sqlite3.Error as e:
    print(f"Erreur SQLite : {str(e)}")
finally:
    print("Traitement terminé. La base de données a été mise à jour avec les thématiques.")

In [ ]:
# Dossier de la base de données SQLite
output_dir = "/home/pionner02/Pionner02 UlChris-Project/data"
db_path = os.path.join(output_dir, 'gdelt_benin.db')

# Connexion à la base de données
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Liste des tables à consulter
tables = [
          'events', 
          'mentions'
          ]

# Explorer les tables
for table in tables:
    try:
        # Aperçu des lignes
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"Nombre de lignes dans {table} : {count}")
        
        # Aperçu des colonnes
        cursor.execute(f"PRAGMA table_info({table})")
        columns = [info[1] for info in cursor.fetchall()]
        print(f"Colonnes dans {table} : {columns}\n")

    except sqlite3.OperationalError as e:
        print(f" Erreur avec la table '{table}' : {e}\n")

# Afficher les index existants dans la base
cursor.execute("SELECT name, tbl_name FROM sqlite_master WHERE type='index'")
indexes = cursor.fetchall()

if indexes:
    print(" Index existants :")
    for idx in indexes:
        print(f" - {idx[0]} (table : {idx[1]})")
else:
    print(" Aucun index existant trouvé.")
conn.close()
print("\n Connexion à la base fermée.")

In [ ]:
# Importer les bibliothèques nécessaires
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from contextlib import closing

# Connexion avec sqlalchemy pour %sql et pandas
engine = create_engine(f"sqlite:///{db_path}")

# Fonction pour récupérer un aperçu de la table
def preview_table(table_name, limit=10):
    query = f"SELECT * FROM {table_name} LIMIT {limit}"
    return pd.read_sql(query, engine)

# Fonction pour afficher la structure de la table
def show_table_structure(table_name):
    query = f"PRAGMA table_info({table_name})"
    return pd.read_sql(query, engine)

# Aperçu des 5 premières lignes de la table 'events'
print("Aperçu de la table 'events' :")
events_df = preview_table('events')
display(events_df)

# Structure de la table 'events' (noms des colonnes et types)
print("\nStructure de la table 'events' :")
events_structure = show_table_structure('events')
display(events_structure)

# Aperçu des 5 premières lignes de la table 'mentions'
print("\nAperçu de la table 'mentions' :")
mentions_df = preview_table('mentions')
display(mentions_df)

# Structure de la table 'mentions' (noms des colonnes et types)
print("\nStructure de la table 'mentions' :")
mentions_structure = show_table_structure('mentions')
display(mentions_structure)

# Fermeture de l'engine SQLAlchemy
engine.dispose()


In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sqlalchemy import create_engine

# Connexion avec sqlalchemy pour %sql et pandas
engine = create_engine(f"sqlite:///{db_path}")

# Statistiques pour la table 'events'

print("=== Statistiques descriptives pour la table 'events' ===")

## Charger les données de 'events' avec SQLAlchemy
events_df = pd.read_sql_query("SELECT * FROM events", engine)

## Stats pour les colonnes numériques
numeric_cols_events = ['GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 
                      'AvgTone']
events_numeric_stats = events_df[numeric_cols_events].describe()
print("\nStatistiques pour les colonnes numériques :")
display(events_numeric_stats)

## Stats pour les colonnes textuelles
non_numeric_cols_events = ['ActionGeo_FullName', 'Themes']
for col in non_numeric_cols_events:
    print(f"\nFréquence des valeurs uniques pour '{col}' :")
    value_counts = events_df[col].value_counts().head(10)
    display(value_counts)

## Événements majeurs
print("\nTop 10 événements par nombre de mentions :")
top_events = events_df.nlargest(10, 'NumMentions')[['GLOBALEVENTID', 'SQLDATE', 'NumMentions', 'AvgTone']]
print(top_events)

print("\nTop 10 événements par impact :")
top_impact = events_df.nlargest(10, 'GoldsteinScale')[['GLOBALEVENTID', 'SQLDATE', 'GoldsteinScale', 'AvgTone']]
print(top_impact)

# Statistiques pour la table 'mentions'

print("\n=== Statistiques descriptives pour la table 'mentions' ===")

## Charger les données de 'mentions' avec SQLAlchemy
mentions_df = pd.read_sql_query("SELECT * FROM mentions", engine)

## Stats pour les colonnes numériques
numeric_cols_mentions = ['Confidence', 'MentionDocTone']
mentions_numeric_stats = mentions_df[numeric_cols_mentions].describe()
print("\nStatistiques pour les colonnes numériques :")
display(mentions_numeric_stats)

## Stats pour les colonnes non numériques
non_numeric_cols_mentions = ['GLOBALEVENTID','MentionType', 'MentionSourceName']
for col in non_numeric_cols_mentions:
    print(f"\nFréquence des valeurs uniques pour '{col}' :")
    value_counts = mentions_df[col].value_counts().head(10)
    display(value_counts)

# Fermeture de l'engine SQLAlchemy
engine.dispose()

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Configurer le style des graphes
plt.style.use('seaborn-v0_8')
sns.set_palette("deep")

# Connexion avec sqlalchemy pour %sql et pandas
engine = create_engine(f"sqlite:///{db_path}")

# Analyse temporelle pour la table 'events'

print("=== Analyse temporelle pour la table 'events' ===")

## Fréquence des événements par mois
events_freq_query = """
SELECT substr(SQLDATE, 1, 6) AS month, COUNT(*) AS event_count
FROM events
GROUP BY month
ORDER BY month
"""
events_freq_df = pd.read_sql_query(events_freq_query, engine)

## Graphique : Fréquence des événements par mois
plt.figure(figsize=(10, 6))
sns.lineplot(data=events_freq_df, x='month', y='event_count', marker='o')
plt.title("Nombre d'événements par mois")
plt.xlabel("Mois (YYYYMM)")
plt.ylabel("Nombre d'événements")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Évolution de GoldsteinScale, NumMentions, NumSources, NumArticles
events_metrics_query = """
SELECT substr(SQLDATE, 1, 6) AS month,
       AVG(GoldsteinScale) AS avg_goldstein,
       SUM(NumMentions) AS total_mentions,
       SUM(NumSources) AS total_sources,
       SUM(NumArticles) AS total_articles
FROM events
GROUP BY month
ORDER BY month
"""
events_metrics_df = pd.read_sql_query(events_metrics_query, engine)

## Graphique : Évolution des métriques
fig, axes = plt.subplots(3, 2, figsize=(14, 10), sharex=True)
fig.suptitle("Évolution des métriques par mois (events)")

## GoldsteinScale
sns.lineplot(data=events_metrics_df, x='month', y='avg_goldstein', marker='o', ax=axes[0, 0])
axes[0, 0].set_title("Moyenne GoldsteinScale")
axes[0, 0].set_ylabel("Moyenne")

## Évolution de NumMentions
sns.lineplot(data=events_metrics_df, x='month', y='total_mentions', marker='o', ax=axes[0, 1])
axes[0, 1].set_title("Total NumMentions")
axes[0, 1].set_ylabel("Total")

## Évolution de NumSources
sns.lineplot(data=events_metrics_df, x='month', y='total_sources', marker='o', ax=axes[1, 0])
axes[1, 0].set_title("Total NumSources")
axes[1, 0].set_ylabel("Total")

## Évolution de NumArticles
sns.lineplot(data=events_metrics_df, x='month', y='total_articles', marker='o', ax=axes[1, 1])
axes[0, 1].set_title("Total NumArticles")
axes[0, 1].set_ylabel("Total")

for ax in axes.flat:
    ax.set_xlabel("Mois (YYYYMM)")
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Analyse temporelle pour la table 'mentions'

print("\n=== Analyse temporelle pour la table 'mentions' ===")

## Fréquence des mentions par mois
mentions_freq_query = """
SELECT substr(MentionTimeDate, 1, 6) AS month, COUNT(*) AS mention_count
FROM mentions
GROUP BY month
ORDER BY month
"""
mentions_freq_df = pd.read_sql_query(mentions_freq_query, engine)

## Graphique : Fréquence des mentions par mois
plt.figure(figsize=(10, 6))
sns.lineplot(data=mentions_freq_df, x='month', y='mention_count', marker='o')
plt.title("Nombre de mentions par mois")
plt.xlabel("Mois (YYYYMM)")
plt.ylabel("Nombre de mentions")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Évolution de MentionDocTone, Confidence
mentions_metrics_query = """
SELECT substr(MentionTimeDate, 1, 6) AS month,
       AVG(MentionDocTone) AS avg_doctone,
       AVG(Confidence) AS avg_confidence
FROM mentions
GROUP BY month
ORDER BY month
"""
mentions_metrics_df = pd.read_sql_query(mentions_metrics_query, engine)

# Évolution des métriques
fig, axes = plt.subplots(2, 1, figsize=(10, 12), sharex=True)
fig.suptitle("Évolution des métriques par mois (mentions)")

# Évolution de MentionDocTone
sns.lineplot(data=mentions_metrics_df, x='month', y='avg_doctone', marker='o', ax=axes[0])
axes[0].set_title("Moyenne MentionDocTone")
axes[0].set_ylabel("Moyenne")

# Évolution de Confidence
sns.lineplot(data=mentions_metrics_df, x='month', y='avg_confidence', marker='o', ax=axes[1])
axes[1].set_title("Moyenne Confidence")
axes[1].set_ylabel("Moyenne")

for ax in axes:
    ax.set_xlabel("Mois (YYYYMM)")
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Fermeture de l'engine SQLAlchemy
engine.dispose()

In [ ]:
# Connexion avec sqlalchemy pour %sql et pandas
engine = create_engine(f"sqlite:///{db_path}")

# Chargement des tables dans les DataFrames
events_df = pd.read_sql_query("SELECT * FROM events", engine)
mentions_df = pd.read_sql_query("SELECT * FROM mentions", engine)

# Conversion de SQLDATE en datetime
events_df['SQLDATE'] = pd.to_datetime(events_df['SQLDATE'], errors='coerce')

## Sentiments : Distribution
plt.figure(figsize=(12, 6))
sns.histplot(events_df['AvgTone'], bins=30, kde=True, label='Events AvgTone')
sns.histplot(mentions_df['MentionDocTone'], bins=30, kde=True, label='Mentions DocTone', alpha=0.5)
plt.title("Distribution des sentiments")
plt.xlabel("Tonalité")
plt.legend()
plt.show()


## Sentiments : Evolution mensuel
events_df['Month'] = events_df['SQLDATE'].dt.strftime('%Y')
sentiment_by_month = events_df.groupby('Month')['AvgTone'].mean()

# Trier les mois dans l'ordre chronologique
sentiment_by_month = sentiment_by_month.sort_index()

sentiment_by_month.plot(kind='bar', figsize=(10, 5), color='skyblue')
plt.title("Sentiment moyen par mois au Bénin (Events)")
plt.ylabel("Tonalité moyenne (AvgTone)")
plt.xlabel("Mois")
plt.tight_layout()
plt.show()


# Fermeture de l'engine SQLAlchemy
engine.dispose()

In [ ]:
import numpy as np

# Connexion à la base
conn = sqlite3.connect(db_path)
engine = create_engine(f"sqlite:///{db_path}")

# Chargement des données
print("=== Chargement des données ===")
query = """
SELECT SQLDATE, GoldsteinScale, NumMentions, NumSources, NumArticles, 
       ActionGeo_FullName, Themes, AvgTone
FROM events
"""
df = pd.read_sql_query(query, conn)
df['SQLDATE'] = pd.to_datetime(df['SQLDATE'], format='%Y-%m-%d')


print("=== Feature engineering ===")
df['year'] = df['SQLDATE'].dt.year
df['month'] = df['SQLDATE'].dt.month
df['day'] = df['SQLDATE'].dt.day

# Visualisation de AvgTone
print("=== Visualisation de AvgTone : Historique ===")

# Historique mensuel
df['date_str'] = df['year'].astype(str) + '-' + df['month'].astype(str).str.zfill(2)
monthly_historical = df.groupby('date_str')['AvgTone'].mean().reset_index()

# Graphique Historique de AvgTone
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_historical, x='date_str', y='AvgTone', label='Historique (AvgTone)', color='blue')
plt.title("Évolution de AvgTone (Historique) : Moyenne Mensuelle")
plt.xlabel("Date (YYYY-MM)")
plt.ylabel("AvgTone")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


# Fermeture de l'engine SQLAlchemy
engine.dispose()

In [ ]:
# Connexion avec sqlite3
conn = sqlite3.connect(db_path)

# Connexion avec sqlalchemy pour %sql
engine = create_engine(f"sqlite:///{db_path}")

# Chargement et Préparation des données

print("=== Chargement et agrégation des données ===")

query_events = """
SELECT GLOBALEVENTID, AvgTone, Themes, ActionGeo_FullName, SQLDATE
FROM events
LIMIT 100
"""
df_events = pd.read_sql_query(query_events, conn)

# Convertir SQLDATE en datetime
df_events['SQLDATE'] = pd.to_datetime(df_events['SQLDATE'], format='%Y-%m-%d')

# Extraire année et mois
df_events['year'] = df_events['SQLDATE'].dt.year
df_events['month'] = df_events['SQLDATE'].dt.month

# Agréger MentionDocTone par GLOBALEVENTID
query_mentions = """
SELECT GLOBALEVENTID,
       MentionDocTone
FROM mentions
WHERE GLOBALEVENTID IN (SELECT GLOBALEVENTID FROM events LIMIT 100)
"""
df_mentions = pd.read_sql_query(query_mentions, conn)

# Vérifier si des mentions ont été trouvées
print("Nombre de mentions trouvées :", len(df_mentions))
print("Nombre de GLOBALEVENTID uniques dans mentions :", df_mentions['GLOBALEVENTID'].nunique())

# Calculer les statistiques de MentionDocTone avec Pandas
if not df_mentions.empty:
    mentions_stats = df_mentions.groupby('GLOBALEVENTID').agg({
        'MentionDocTone': ['mean', 'min', 'max', 'std']
    }).reset_index()
    # Renommer les colonnes
    mentions_stats.columns = ['GLOBALEVENTID', 'avg_mention_tone', 'min_mention_tone', 'max_mention_tone', 'std_mention_tone']
else:
    print("Aucune mention trouvée pour les GLOBALEVENTID sélectionnés.")
    # Créer un DataFrame vide avec les colonnes attendues
    mentions_stats = pd.DataFrame(columns=['GLOBALEVENTID', 'avg_mention_tone', 'min_mention_tone', 'max_mention_tone', 'std_mention_tone'])

# Joindre les deux DataFrames
df = df_events.merge(mentions_stats, on='GLOBALEVENTID', how='left')

# Vérifier les valeurs manquantes
print("Valeurs manquantes :")
print(df.isnull().sum())

# Remplacer les valeurs manquantes
df['avg_mention_tone'] = df['avg_mention_tone'].fillna(0)
df['min_mention_tone'] = df['min_mention_tone'].fillna(0)
df['max_mention_tone'] = df['max_mention_tone'].fillna(0)
df['std_mention_tone'] = df['std_mention_tone'].fillna(0)
df['Themes'] = df['Themes'].fillna("Inconnu")
df['ActionGeo_FullName'] = df['ActionGeo_FullName'].fillna("Inconnu")

# Supprimer les lignes avec AvgTone ou SQLDATE manquant
df = df.dropna(subset=['AvgTone', 'SQLDATE'])

# Analyse des sentiments avec call_openai_api
print("=== Analyse des sentiments ===")

# Fonction pour analyser le sentiment d'un événement
def analyze_sentiment(row):
    avg_tone = row['AvgTone']
    themes = row['Themes']
    location = row['ActionGeo_FullName']
    avg_mention_tone = row['avg_mention_tone']
    min_mention_tone = row['min_mention_tone']
    max_mention_tone = row['max_mention_tone']
    std_mention_tone = row['std_mention_tone']
    year = row['year']
    month = row['month']
    
    # Prompt pour analyse des sentimeents
    prompt = f"""
    Vous êtes un expert en analyse des données issues de GDELT (Global Database of Events, Language, and Tone), une base de données mondiale sur les événements médiatisés. Votre tâche est d'analyser le sentiment d'un événement en fonction de son AvgTone, de ses thèmes, de sa localisation, des tons des mentions individuelles (MentionDocTone), et de sa date.

    Données de l'événement :
    - AvgTone : {avg_tone} (score numérique, généralement entre -15 et +15, où négatif indique un ton défavorable, positif un ton favorable, et près de 0 un ton neutre).
    - Thèmes : {themes} (catégorie décrivant la nature de l'événement, ex. POLITIQUE, CONFLIT, CÉLÉBRATION).
    - Localisation : {location} (lieu géographique de l'événement).
    - MentionDocTone (ton des articles individuels) :
      - Moyenne : {avg_mention_tone}
      - Minimum : {min_mention_tone}
      - Maximum : {max_mention_tone}
      - Écart-type : {std_mention_tone} (indique la variabilité des tons des mentions).
    - Date : Année {year}, Mois {month} (contexte temporel de l'événement).

    Instructions :
    1. Déterminez le sentiment de l'événement : "Positif", "Neutre" ou "Négatif".
    2. Fournissez une explication concise traduidant pourquoi ce sentiment a été attribué, en tenant compte du contexte GDELT (ex. biais médiatiques, nature des thèmes, impact de la localisation, variabilité des MentionDocTone, et contexte temporel).
    3. Comparez AvgTone et MentionDocTone pour identifier d'éventuelles incohérences ou polarisations, et mentionnez l'influence possible de la date (ex. événements mondiaux majeurs à cette période).
    4. Structurez la réponse comme suit :
       Sentiment : [Positif/Neutre/Négatif]
       Explication : [Votre explication]

    Exemple :
    Sentiment : Positif
    Explication : Un AvgTone de 5.0 et une moyenne MentionDocTone de 4.8 indiquent un ton médiatique favorable, renforcé par le thème CÉLÉBRATION en décembre 2020, probablement lié à des festivités de fin d'année. La faible variabilité (écart-type de 0.5) suggère une couverture cohérente.
    """
    
    # Appeler l'API
    try:
        response = call_openai_api(prompt, max_tokens=500)
        # Extraire le sentiment et l'explication
        sentiment = response.split("Sentiment : ")[1].split("\n")[0].strip()
        explanation = response.split("Explication : ")[1].strip()
        return pd.Series([sentiment, explanation], index=['Sentiment', 'Explanation'])
    except Exception as e:
        print(f"Erreur pour GLOBALEVENTID {row['GLOBALEVENTID']}: {e}")
        return pd.Series(['Inconnu', f'Erreur API: {str(e)}'], index=['Sentiment', 'Explanation'])

# Appliquer l'analyse à chaque ligne
df[['Sentiment', 'Explanation']] = df.apply(analyze_sentiment, axis=1)

# Résumé des sentiments
print("=== Résumé des sentiments ===")

# Compter les sentiments
sentiment_counts = df['Sentiment'].value_counts()
print("\nRépartition des sentiments :")
print(sentiment_counts)

# Moyenne de AvgTone et avg_mention_tone par sentiment
sentiment_summary = df.groupby('Sentiment')[['AvgTone', 'avg_mention_tone']].mean()
print("\nMoyennes de AvgTone et avg_mention_tone par sentiment :")
print(sentiment_summary)

# Analyse temporelle

print("=== Analyse temporelle ===")

# Répartition des sentiments par année
sentiment_by_year = df.groupby(['year', 'Sentiment']).size().unstack(fill_value=0)
sentiment_by_year = sentiment_by_year.div(sentiment_by_year.sum(axis=1), axis=0)  # Normaliser en proportions
print("\nRépartition des sentiments par année (proportions) :")
print(sentiment_by_year)

# Moyenne de AvgTone et avg_mention_tone par année
tone_by_year = df.groupby('year')[['AvgTone', 'avg_mention_tone']].mean()
print("\nMoyenne de AvgTone et avg_mention_tone par année :")
print(tone_by_year)

# Visualisation des sentiments
print("=== Visualisation ===")

# Répartition des sentiments par année
plt.figure(figsize=(10, 6))
sentiment_by_year.plot(kind='bar', stacked=True, colormap='viridis')
plt.title("Répartition des sentiments par année (GDELT)")
plt.xlabel("Année")
plt.ylabel("Proportion")
plt.legend(title="Sentiment")
plt.tight_layout()
plt.show()

# Évolution de AvgTone et avg_mention_tone par mois
df['year_month'] = df['SQLDATE'].dt.to_period('M')
tone_by_month = df.groupby('year_month')[['AvgTone', 'avg_mention_tone']].mean()
plt.figure(figsize=(12, 6))
tone_by_month.plot()
plt.title("Évolution de AvgTone et MentionDocTone moyen par mois (GDELT)")
plt.xlabel("Date (YYYY-MM)")
plt.ylabel("Ton")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Répartition des sentiments
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Sentiment', order=['Positif', 'Neutre', 'Négatif', 'Inconnu'])
plt.title("Répartition des sentiments des événements (GDELT)")
plt.xlabel("Sentiment")
plt.ylabel("Nombre d'événements")
plt.tight_layout()
plt.show()

# Graphique : AvgTone vs avg_mention_tone par sentiment
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df, x='AvgTone', y='avg_mention_tone', hue='Sentiment', style='Sentiment')
plt.title("AvgTone vs MentionDocTone moyen par sentiment (GDELT)")
plt.xlabel("AvgTone")
plt.ylabel("MentionDocTone moyen")
plt.tight_layout()
plt.show()

# Répartition des sentiments par mois (en proportion)
sentiment_by_month = df.groupby(['year_month', 'Sentiment']).size().unstack(fill_value=0)
sentiment_by_month = sentiment_by_month.div(sentiment_by_month.sum(axis=1), axis=0)

# Evolution des sentiments dans le temps
plt.figure(figsize=(12, 6))
sentiment_by_month.plot(marker='o')
plt.title("Évolution temporelle des sentiments (par mois)")
plt.xlabel("Date (YYYY-MM)")
plt.ylabel("Proportion (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Créer une matrice année/mois pour AvgTone
pivot_heatmap = df.pivot_table(values='AvgTone', index='year', columns='month', aggfunc='mean')

# Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_heatmap, annot=True, fmt=".1f", cmap="coolwarm", center=0)
plt.title("Carte thermique de AvgTone (année vs mois)")
plt.xlabel("Mois")
plt.ylabel("Année")
plt.tight_layout()
plt.show()

# Extrait les 5 thèmes les plus fréquents
top_themes = df['Themes'].value_counts().head(5).index

# Courbes d’AvgTone par thème
plt.figure(figsize=(12, 6))
for theme in top_themes:
    subset = df[df['Themes'] == theme]
    tone_by_theme = subset.groupby('year_month')['AvgTone'].mean()
    plt.plot(tone_by_theme.index.astype(str), tone_by_theme.values, label=theme)


# Sauvegarde du DataFrame avec les sentiments
print("=== Sauvegarde des résultats ===")

# Dossier de destination de l'analyse des sentiments
csv_output = "/home/pionner02/Pionner02 UlChris-Project/data/event_sentiment_analysis_gdelt_with_mentions_temporal.csv"

df.to_csv(csv_output, index=False, encoding="utf-8")
print("Résultats sauvegardés dans 'event_sentiment_analysis_gdelt_with_mentions_temporal.csv'")

# Fermeture de la connexion
conn.close()

## Dashboard

Ce tableau de bord vous propose une exploration interactive de ces données, combinant événements et mentions médiatiques, enrichies par des filtres temporels, géographiques et thématiques.

In [3]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import sqlite3
from datetime import datetime
import os

# Chemin vers la base de données
output_dir = "/home/pionner02/Pionner02 UlChris-Project/data"
db_path = os.path.join(output_dir, 'gdelt_benin.db')

# Connexion à la base SQLite
conn = sqlite3.connect(db_path)

# Chargement des données
query_events = """
SELECT GLOBALEVENTID, SQLDATE, GoldsteinScale, NumMentions, NumSources, NumArticles, AvgTone,
       ActionGeo_FullName, ActionGeo_Lat, ActionGeo_Long, Themes
FROM events
WHERE ActionGeo_Lat IS NOT NULL AND ActionGeo_Long IS NOT NULL
LIMIT 10000  -- Limiter pour accélérer
"""
query_mentions = """
SELECT GLOBALEVENTID, MentionTimeDate, MentionSourceName, MentionIdentifier, Confidence, MentionDocTone
FROM mentions
LIMIT 10000  -- Limiter pour accélérer
"""
df_events = pd.read_sql_query(query_events, conn)
df_mentions = pd.read_sql_query(query_mentions, conn)
conn.close()

# Débogage des données
print("df_events shape:", df_events.shape)
print("df_mentions shape:", df_mentions.shape)
print("df_events null counts:\n", df_events.isnull().sum())
print("df_mentions null counts:\n", df_mentions.isnull().sum())

# Nettoyage des données
df_events['Themes'] = df_events['Themes'].fillna('').astype(str)
df_events['NumMentions'] = pd.to_numeric(df_events['NumMentions'], errors='coerce').fillna(0)
df_events['NumSources'] = pd.to_numeric(df_events['NumSources'], errors='coerce').fillna(0)
df_events['NumArticles'] = pd.to_numeric(df_events['NumArticles'], errors='coerce').fillna(0)
df_events['GoldsteinScale'] = pd.to_numeric(df_events['GoldsteinScale'], errors='coerce').fillna(0)
df_events['AvgTone'] = pd.to_numeric(df_events['AvgTone'], errors='coerce').fillna(0)
df_mentions['Confidence'] = pd.to_numeric(df_mentions['Confidence'], errors='coerce').fillna(0)
df_mentions['MentionDocTone'] = pd.to_numeric(df_mentions['MentionDocTone'], errors='coerce').fillna(0)

# Conversion des dates
df_events['SQLDATE'] = pd.to_datetime(df_events['SQLDATE'], format='%Y-%m-%d', errors='coerce')
df_mentions['MentionTimeDate'] = pd.to_datetime(df_mentions['MentionTimeDate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Hiérarchies de dates
df_events['Year'] = df_events['SQLDATE'].dt.year
df_events['Quarter'] = df_events['SQLDATE'].dt.quarter
df_events['Month'] = df_events['SQLDATE'].dt.month
df_events['Week'] = df_events['SQLDATE'].dt.isocalendar().week
df_events['Day'] = df_events['SQLDATE'].dt.day

df_mentions['Year'] = df_mentions['MentionTimeDate'].dt.year
df_mentions['Quarter'] = df_mentions['MentionTimeDate'].dt.quarter
df_mentions['Month'] = df_mentions['MentionTimeDate'].dt.month
df_mentions['Week'] = df_mentions['MentionTimeDate'].dt.isocalendar().week
df_mentions['Day'] = df_mentions['MentionTimeDate'].dt.day

# Fusionner les données pour le tableau
df_merged = pd.merge(df_events, df_mentions, on='GLOBALEVENTID', how='left')  # Utiliser left pour éviter vide
df_merged.fillna({'Themes': '', 'MentionSourceName': '', 'MentionIdentifier': '', 'Confidence': 0, 'MentionDocTone': 0}, inplace=True)
print("df_merged shape:", df_merged.shape)
print("df_merged head:\n", df_merged.head())
print("df_merged null counts:\n", df_merged.isnull().sum())

# Options pour les filtres
years = sorted(df_events['Year'].dropna().unique())
quarters = [1, 2, 3, 4]
months = sorted(df_events['Month'].dropna().unique())
weeks = sorted(df_events['Week'].dropna().unique())
themes = pd.Series(df_events['Themes'].str.split(';', expand=True).stack().unique()).dropna()
sources = df_mentions['MentionSourceName'].dropna().unique()

# Initialisation de l'application Dash
app = dash.Dash(__name__)
app.scripts.config.serve_locally = True
app.css.config.serve_locally = True

# Style CSS pour les cadres
box_style = {
    'border': '1px solid #ccc',
    'borderRadius': '5px',
    'padding': '10px',
    'margin': '10px',
    'textAlign': 'center',
    'boxShadow': '2px 2px 5px rgba(0,0,0,0.1)',
    'width': '200px',
    'display': 'inline-block'
}

# Layout du dashboard
app.layout = html.Div([
    html.H1('Dashboard des Événements et Mentions', style={'textAlign': 'center'}),

    # Filtres
    html.Div([
        html.H3('Filtres SQLDATE'),
        html.Label('Année'),
        dcc.Dropdown(id='sql-year', options=[{'label': y, 'value': y} for y in years], multi=True, style={'width': '50%'}),
        html.Label('Trimestre'),
        dcc.Dropdown(id='sql-quarter', options=[{'label': q, 'value': q} for q in quarters], multi=True, style={'width': '50%'}),
        html.Label('Mois'),
        dcc.Dropdown(id='sql-month', options=[{'label': m, 'value': m} for m in months], multi=True, style={'width': '50%'}),
        html.Label('Semaine'),
        dcc.Dropdown(id='sql-week', options=[{'label': w, 'value': w} for w in weeks], multi=True, style={'width': '50%'}),
    ], style={'width': '45%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Div([
        html.H3('Filtres MentionTimeDate'),
        html.Label('Année'),
        dcc.Dropdown(id='mention-year', options=[{'label': y, 'value': y} for y in years], multi=True, style={'width': '50%'}),
        html.Label('Trimestre'),
        dcc.Dropdown(id='mention-quarter', options=[{'label': q, 'value': q} for q in quarters], multi=True, style={'width': '50%'}),
        html.Label('Mois'),
        dcc.Dropdown(id='mention-month', options=[{'label': m, 'value': m} for m in months], multi=True, style={'width': '50%'}),
        html.Label('Semaine'),
        dcc.Dropdown(id='mention-week', options=[{'label': w, 'value': w} for w in weeks], multi=True, style={'width': '50%'}),
    ], style={'width': '45%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Div([
        html.H3('Filtres Additionnels'),
        html.Label('Thèmes'),
        dcc.Dropdown(id='themes-filter', options=[{'label': t, 'value': t} for t in themes], multi=True, style={'width': '50%'}),
        html.Label('Source de Mention'),
        dcc.Dropdown(id='source-filter', options=[{'label': s, 'value': s} for s in sources], multi=True, style={'width': '50%'}),
    ], style={'margin': '20px'}),

    # Tableau principal
    html.H3('Tableau des Données'),
    dash_table.DataTable(
        id='data-table',
        columns=[
            {'name': 'GLOBALEVENTID', 'id': 'GLOBALEVENTID'},
            {'name': 'GoldsteinScale', 'id': 'GoldsteinScale'},
            {'name': 'NumMentions', 'id': 'NumMentions'},
            {'name': 'NumSources', 'id': 'NumSources'},
            {'name': 'NumArticles', 'id': 'NumArticles'},
            {'name': 'AvgTone', 'id': 'AvgTone'},
            {'name': 'MentionSourceName', 'id': 'MentionSourceName'},
            {'name': 'MentionIdentifier', 'id': 'MentionIdentifier'},
            {'name': 'Confidence', 'id': 'Confidence'},
            {'name': 'MentionDocTone', 'id': 'MentionDocTone'},
        ],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'maxWidth': '300px', 'whiteSpace': 'normal'},
        data=df_merged[[
            'GLOBALEVENTID', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles',
            'AvgTone', 'MentionSourceName', 'MentionIdentifier', 'Confidence', 'MentionDocTone'
        ]].head(100).to_dict('records'),
    ),

    # Carte géographique
    html.H3('Carte des Événements'),
    dcc.Graph(id='geo-map'),

    # Cadres pour NumMentions, NumSources, NumArticles
    html.H3('Métriques des Mentions, Sources et Articles'),
    html.Div([
        html.Div([
            html.H4('Total Mentions'),
            html.P(id='num-mentions', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
        html.Div([
            html.H4('Total Sources'),
            html.P(id='num-sources', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
        html.Div([
            html.H4('Total Articles'),
            html.P(id='num-articles', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
    ], style={'textAlign': 'center'}),

    # Tableau pour Top 5 Themes
    html.H3('Top 5 Thèmes avec AvgTone'),
    dash_table.DataTable(
        id='themes-table',
        columns=[
            {'name': 'Thème', 'id': 'Theme'},
            {'name': 'Occurrences', 'id': 'Count'},
            {'name': 'AvgTone Moyen', 'id': 'AvgTone'}
        ],
        page_size=5,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'maxWidth': '300px', 'whiteSpace': 'normal'},
        data=df_merged['Themes'].str.split(';', expand=True).stack().value_counts().head(5).reset_index().rename(columns={'index': 'Theme', 'count': 'Count'}).assign(AvgTone=0).to_dict('records'),  # Données par défaut
    ),

    # Cadres pour moyennes des métriques
    html.H3('Moyennes des Métriques'),
    html.Div([
        html.Div([
            html.H4('Moyenne GoldsteinScale'),
            html.P(id='avg-goldstein', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
        html.Div([
            html.H4('Moyenne AvgTone'),
            html.P(id='avg-tone', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
        html.Div([
            html.H4('Moyenne Confidence'),
            html.P(id='avg-confidence', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
        html.Div([
            html.H4('Moyenne MentionDocTone'),
            html.P(id='avg-mention-tone', style={'fontSize': '20px', 'fontWeight': 'bold'})
        ], style=box_style),
    ], style={'textAlign': 'center'}),
])

# Callback pour mettre à jour tous les composants
@app.callback(
    [Output('data-table', 'data'),
     Output('geo-map', 'figure'),
     Output('num-mentions', 'children'),
     Output('num-sources', 'children'),
     Output('num-articles', 'children'),
     Output('themes-table', 'data'),
     Output('avg-goldstein', 'children'),
     Output('avg-tone', 'children'),
     Output('avg-confidence', 'children'),
     Output('avg-mention-tone', 'children')],
    [Input('sql-year', 'value'),
     Input('sql-quarter', 'value'),
     Input('sql-month', 'value'),
     Input('sql-week', 'value'),
     Input('mention-year', 'value'),
     Input('mention-quarter', 'value'),
     Input('mention-month', 'value'),
     Input('mention-week', 'value'),
     Input('themes-filter', 'value'),
     Input('source-filter', 'value')]
)
def update_dashboard(sql_year, sql_quarter, sql_month, sql_week,
                    mention_year, mention_quarter, mention_month, mention_week,
                    themes, sources):
    filtered_df = df_merged.copy()
    
    print("Initial filtered_df shape:", filtered_df.shape)
    
    # Filtres SQLDATE
    if sql_year:
        filtered_df = filtered_df[filtered_df['Year_x'].isin(sql_year) & filtered_df['Year_x'].notna()]
    if sql_quarter:
        filtered_df = filtered_df[filtered_df['Quarter_x'].isin(sql_quarter) & filtered_df['Quarter_x'].notna()]
    if sql_month:
        filtered_df = filtered_df[filtered_df['Month_x'].isin(sql_month) & filtered_df['Month_x'].notna()]
    if sql_week:
        filtered_df = filtered_df[filtered_df['Week_x'].isin(sql_week) & filtered_df['Week_x'].notna()]
    
    # Filtres MentionTimeDate
    if mention_year:
        filtered_df = filtered_df[filtered_df['Year_y'].isin(mention_year) & filtered_df['Year_y'].notna()]
    if mention_quarter:
        filtered_df = filtered_df[filtered_df['Quarter_y'].isin(mention_quarter) & filtered_df['Quarter_y'].notna()]
    if mention_month:
        filtered_df = filtered_df[filtered_df['Month_y'].isin(mention_month) & filtered_df['Month_y'].notna()]
    if mention_week:
        filtered_df = filtered_df[filtered_df['Week_y'].isin(mention_week) & filtered_df['Week_y'].notna()]
    
    # Filtres Thèmes
    if themes:
        filtered_df = filtered_df[filtered_df['Themes'].str.contains('|'.join(themes), na=False)]
    
    # Filtres Sources
    if sources:
        filtered_df = filtered_df[filtered_df['MentionSourceName'].isin(sources) & filtered_df['MentionSourceName'].notna()]
    
    print("Final filtered_df shape:", filtered_df.shape)
    print("Final filtered_df head:\n", filtered_df.head())
    
    # Tableau principal
    table_data = filtered_df[[
        'GLOBALEVENTID', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles',
        'AvgTone', 'MentionSourceName', 'MentionIdentifier', 'Confidence', 'MentionDocTone'
    ]].to_dict('records')
    
    # Carte géographique
    fig_map = px.scatter_geo(
        filtered_df,
        lat='ActionGeo_Lat',
        lon='ActionGeo_Long',
        hover_name='ActionGeo_FullName',
        size='NumMentions',
        color='AvgTone',
        color_continuous_scale='RdBu',
        title='Carte des Événements',
        projection='natural earth'
    )
    fig_map.update_layout(
        geo=dict(showframe=False, showcoastlines=True),
        dragmode='zoom',
        margin={'l': 0, 'r': 0, 't': 50, 'b': 0}
    )
    
    # Sommes pour NumMentions, NumSources, NumArticles
    total_mentions = f"{int(filtered_df['NumMentions'].sum()):,}" if not filtered_df['NumMentions'].empty else "0"
    total_sources = f"{int(filtered_df['NumSources'].sum()):,}" if not filtered_df['NumSources'].empty else "0"
    total_articles = f"{int(filtered_df['NumArticles'].sum()):,}" if not filtered_df['NumArticles'].empty else "0"
    
    # Tableau pour Top 5 Themes avec AvgTone
    if not filtered_df.empty and filtered_df['Themes'].str.strip().ne('').any():
        themes_data = filtered_df['Themes'].str.split(';', expand=True).stack().reset_index(drop=True)
        themes_counts = themes_data.value_counts().head(5).reset_index()
        themes_counts.columns = ['Theme', 'Count']
        themes_avg_tone = []
        for theme in themes_counts['Theme']:
            theme_mask = filtered_df['Themes'].str.contains(theme, na=False)
            avg_tone = filtered_df[theme_mask]['AvgTone'].mean() if theme_mask.any() else 0
            themes_avg_tone.append(round(avg_tone, 2) if pd.notna(avg_tone) else 0)
        themes_counts['AvgTone'] = themes_avg_tone
    else:
        themes_counts = pd.DataFrame({'Theme': ['Aucun thème'], 'Count': [0], 'AvgTone': [0]})
    themes_table_data = themes_counts.to_dict('records')
    print("Themes table data:", themes_table_data)
    
    # Moyennes des métriques
    avg_goldstein = round(filtered_df['GoldsteinScale'].mean(), 2) if not filtered_df['GoldsteinScale'].empty else 0
    avg_tone = round(filtered_df['AvgTone'].mean(), 2) if not filtered_df['AvgTone'].empty else 0
    avg_confidence = round(filtered_df['Confidence'].mean(), 2) if not filtered_df['Confidence'].empty else 0
    avg_mention_tone = round(filtered_df['MentionDocTone'].mean(), 2) if not filtered_df['MentionDocTone'].empty else 0
    
    return (
        table_data,
        fig_map,
        total_mentions,
        total_sources,
        total_articles,
        themes_table_data,
        str(avg_goldstein),
        str(avg_tone),
        str(avg_confidence),
        str(avg_mention_tone)
    )

# Lancement du serveur
if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=8050)

df_events shape: (10000, 11)
df_mentions shape: (10000, 6)
df_events null counts:
 GLOBALEVENTID            0
SQLDATE                  0
GoldsteinScale           0
NumMentions              0
NumSources               0
NumArticles              0
AvgTone                  0
ActionGeo_FullName       0
ActionGeo_Lat            0
ActionGeo_Long           0
Themes                9180
dtype: int64
df_mentions null counts:
 GLOBALEVENTID        0
MentionTimeDate      0
MentionSourceName    0
MentionIdentifier    0
Confidence           0
MentionDocTone       0
dtype: int64
df_merged shape: (10131, 26)
df_merged head:
    GLOBALEVENTID    SQLDATE  GoldsteinScale  NumMentions  NumSources  \
0     1238864555 2025-04-19           -10.0           15           2   
1     1239269408 2025-04-21           -10.0           16           2   
2     1117208340 2023-07-26            -2.0           30           3   
3     1117210338 2023-07-26             3.4           12           1   
4     1124580946 2023-09

Initial filtered_df shape: (10131, 26)
Final filtered_df shape: (10131, 26)
Final filtered_df head:
    GLOBALEVENTID    SQLDATE  GoldsteinScale  NumMentions  NumSources  \
0     1238864555 2025-04-19           -10.0           15           2   
1     1239269408 2025-04-21           -10.0           16           2   
2     1117208340 2023-07-26            -2.0           30           3   
3     1117210338 2023-07-26             3.4           12           1   
4     1124580946 2023-09-03             3.4           12           2   

   NumArticles   AvgTone              ActionGeo_FullName  ActionGeo_Lat  \
0           15 -9.746639  Koudou, Benin (general), Benin        10.8945   
1           16 -3.942652                           Benin         9.5000   
2           30 -4.848485                           Benin         9.5000   
3           12  4.403409                           Benin         9.5000   
4           12  2.535821                           Benin         9.5000   

   ActionGeo_Lo